In [153]:
# Dependencies and Setup
import os
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [154]:
#read the data sets
csv_path = "Resources/Zillow Observed Rent Index, smoothed, seasonally adjusted.csv"
rent_df = pd.read_csv(csv_path)


In [155]:
csv_path = "Resources/Zillow House Value Index, smoothed, seaonally adjusted.csv"
house_df = pd.read_csv(csv_path)

In [156]:
## rent data cleaning

In [157]:
rent_df.head()

,RegionID,RegionName,SizeRank,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,...,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01
0,102001,United States,0,1373.0,1380.0,1388.0,1395.0,1402.0,1409,1417,...,1744,1745,1745,1746,1746,1747,1747,1747,1747.0,1748.0
1,394913,"New York, NY",1,2379.0,2391.0,2403.0,2415.0,2426.0,2438,2450,...,2756,2734,2713,2691,2669,2646,2624,2600,2576.0,2552.0
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,1831.0,1848.0,1865.0,1882.0,1899.0,1916,1932,...,2566,2565,2564,2563,2561,2559,2558,2556,2554.0,2553.0
3,394463,"Chicago, IL",3,1511.0,1517.0,1522.0,1528.0,1533.0,1539,1544,...,1766,1762,1758,1754,1750,1746,1742,1737,1732.0,1728.0
4,394514,"Dallas-Fort Worth, TX",4,1199.0,1207.0,1214.0,1221.0,1229.0,1236,1243,...,1566,1568,1571,1573,1575,1577,1580,1582,1585.0,1587.0


In [158]:
#depivot the table from the time series
rent_df = pd.melt(rent_df, id_vars=['RegionID', 'RegionName','SizeRank'], var_name='year', value_name='price')

In [222]:
rent_df

,RegionID,RegionName,SizeRank,year,price,Year,Month
0,102001,United States,0,2014-01-01,1373,2014,1
1,394913,"New York, NY",1,2014-01-01,2379,2014,1
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,2014-01-01,1831,2014,1
3,394463,"Chicago, IL",3,2014-01-01,1511,2014,1
4,394514,"Dallas-Fort Worth, TX",4,2014-01-01,1199,2014,1
...,...,...,...,...,...,...,...
9005,394995,"Port St. Lucie, FL",119,2021-01-01,1815,2021,1
9006,394602,"Fort Collins, CO",159,2021-01-01,1618,2021,1
9007,394405,"Boulder, CO",162,2021-01-01,2020,2021,1
9008,394645,"Greeley, CO",177,2021-01-01,1681,2021,1


In [159]:
#check for null values
rent_df.isnull().sum()

RegionID       0
RegionName     0
SizeRank       0
year           0
price         17
dtype: int64

In [160]:
#drop the null values
rent_df = rent_df.dropna()

In [161]:
rent_df.isnull().sum()

RegionID      0
RegionName    0
SizeRank      0
year          0
price         0
dtype: int64

In [162]:
#data types check
rent_df.dtypes

RegionID        int64
RegionName     object
SizeRank        int64
year           object
price         float64
dtype: object

In [163]:
#convert data types for further analysis
rent_df['price'] = rent_df['price'].astype(int)
rent_df['year'] = pd.to_datetime(rent_df['year'])


In [164]:
#rent_df['year'] = rent_df['year'].apply(lambda x:x.strftime('%Y-%m'))

In [165]:
#double check data type
rent_df.dtypes

RegionID               int64
RegionName            object
SizeRank               int64
year          datetime64[ns]
price                  int64
dtype: object

In [166]:
rent_df

,RegionID,RegionName,SizeRank,year,price
0,102001,United States,0,2014-01-01,1373
1,394913,"New York, NY",1,2014-01-01,2379
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,2014-01-01,1831
3,394463,"Chicago, IL",3,2014-01-01,1511
4,394514,"Dallas-Fort Worth, TX",4,2014-01-01,1199
...,...,...,...,...,...
9005,394995,"Port St. Lucie, FL",119,2021-01-01,1815
9006,394602,"Fort Collins, CO",159,2021-01-01,1618
9007,394405,"Boulder, CO",162,2021-01-01,2020
9008,394645,"Greeley, CO",177,2021-01-01,1681


In [217]:
#change the format of date back to its original as the date was not available in the csv
rent_df["Year"] = rent_df.year.dt.year
rent_df["Month"] = rent_df.year.dt.month
#rent_df["Date"] = pd.to_datetime(rent_df[['Year','Month']])

In [209]:
#years data frame
df = rent_df.loc[rent_df["year"] == "2014-01-01"]
df

,RegionID,RegionName,SizeRank,year,price
0,102001,United States,0,2014-01-01,1373
1,394913,"New York, NY",1,2014-01-01,2379
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,2014-01-01,1831
3,394463,"Chicago, IL",3,2014-01-01,1511
4,394514,"Dallas-Fort Worth, TX",4,2014-01-01,1199
...,...,...,...,...,...
101,394995,"Port St. Lucie, FL",119,2014-01-01,1179
102,394602,"Fort Collins, CO",159,2014-01-01,1168
103,394405,"Boulder, CO",162,2014-01-01,1494
104,394645,"Greeley, CO",177,2014-01-01,1167


In [168]:
## housing data cleaning

In [169]:
house_df

,RegionID,SizeRank,RegionName,RegionType,StateName,1996-01-31,1996-02-29,1996-03-31,1996-04-30,1996-05-31,...,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31
0,102001,0,United States,Country,NaN,106811.0,106838.0,106888.0,107014.0,107156.0,...,250069.0,251258.0,252506.0,254019.0,255964.0,258235.0,260729.0,263427.0,266222.0,269039.0
1,394913,1,"New York, NY",Msa,NY,187591.0,187153.0,186875.0,186342.0,186025.0,...,483751.0,485191.0,486992.0,489417.0,492642.0,496858.0,501651.0,506833.0,511880.0,516732.0
2,753899,2,"Los Angeles-Long Beach-Anaheim, CA",Msa,CA,186111.0,186371.0,186390.0,186499.0,186473.0,...,698120.0,699783.0,701018.0,705448.0,712728.0,720482.0,727461.0,734522.0,741757.0,748532.0
3,394463,3,"Chicago, IL",Msa,IL,163099.0,162799.0,162404.0,161955.0,161353.0,...,245195.0,245715.0,246350.0,247476.0,248977.0,250996.0,253227.0,255446.0,257610.0,259459.0
4,394514,4,"Dallas-Fort Worth, TX",Msa,TX,111373.0,111437.0,111596.0,111914.0,112257.0,...,256755.0,257898.0,259226.0,260751.0,262469.0,264207.0,266483.0,268773.0,271372.0,273348.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,394767,929,"Lamesa, TX",Msa,TX,NaN,NaN,NaN,NaN,NaN,...,74888.0,75130.0,75441.0,75725.0,76039.0,76385.0,76733.0,77319.0,78015.0,78762.0
909,753874,930,"Craig, CO",Msa,CO,61884.0,62113.0,62354.0,62824.0,63158.0,...,191181.0,191259.0,191465.0,191633.0,191837.0,192061.0,192424.0,193341.0,194323.0,195374.0
910,394968,931,"Pecos, TX",Msa,TX,NaN,NaN,NaN,NaN,NaN,...,134245.0,134927.0,135804.0,136325.0,136118.0,134957.0,133241.0,131749.0,130777.0,129578.0
911,395188,932,"Vernon, TX",Msa,TX,NaN,NaN,NaN,NaN,NaN,...,68440.0,68423.0,68364.0,68360.0,68365.0,68420.0,68363.0,68378.0,68339.0,68432.0


In [183]:
#depivot the table
house_df = pd.melt(house_df, id_vars=['RegionID', 'RegionName','SizeRank'], var_name='Date', value_name='price')

In [184]:
#check for null values
house_df.isnull().sum()

RegionID      0
RegionName    0
SizeRank      0
Date          0
price         0
dtype: int64

In [185]:
#drop the null values
house_df = house_df.dropna()

In [186]:
house_df.isnull().sum()

RegionID      0
RegionName    0
SizeRank      0
Date          0
price         0
dtype: int64

In [187]:
house_df.dtypes

RegionID       int64
RegionName    object
SizeRank       int64
Date          object
price         object
dtype: object

In [218]:
#convert data types for further analysis
#house_df['price'] = house_df['price'].astype(str).astype(int)
house_df['Date'] = pd.to_datetime(house_df['Date'])

ParserError: Unknown string format: year

In [ ]:
#rent_df.to_csv('Resources'\adjusted_rent.csv')